In [0]:
!pip install eli5

     |████████████████████████████████| 112kB 2.9MB/s 


In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance
from ast import literal_eval
from tqdm import tqdm_notebook

In [0]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix"

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [0]:
ls data

shoes_prices2.csv  shoes_prices.csv


In [0]:
df = pd.read_csv('data/shoes_prices2.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  X = df[feats].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model,X,y,scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [0]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]

In [0]:
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [0]:
def run_model_forest(feats, model = RandomForestRegressor(max_depth=5,n_estimators=100,random_state=0)):
  X = df[feats].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model,X,y,scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [0]:
run_model_forest(['brand_cat'])

(-57.31783843165656, 4.181246596160967)

In [0]:
df.features.head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [0]:
test = {'key' : 'value'}
test['key']
str(test)

"{'key': 'value'}"

In [0]:
str_dict = '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]'

literal_eval(str_dict)

[{'key': 'Gender', 'value': ['Men']},
 {'key': 'Shoe Size', 'value': ['M']},
 {'key': 'Shoe Category', 'value': ["Men's Shoes"]},
 {'key': 'Color', 'value': ['Multicolor']},
 {'key': 'Manufacturer Part Number', 'value': ['8190-W-NAVY-7.5']},
 {'key': 'Brand', 'value': ['Josmo']}]

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan':return output_dict
  features = literal_eval(x.replace('\\"',' '))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output_dict[key] = value

  return output_dict

df['features_parsed'] = df['features'].map(parse_features)

In [0]:
keys = set()
df['features_parsed'].map(lambda x: keys.update(x.keys()))
len(keys)

476

In [0]:
def get_name_feat(key):
  return 'feat_'+ key


for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

In [0]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_clasp type', 'feat_protection', 'feat_chain included',
       'feat_vehicle type', 'feat_assembled in country of origin',
       'feat_labor', 'feat_technician', 'feat_digital camera',
       'feat_chronograph', 'feat_water resistant'],
      dtype='object', length=526)

In [0]:
keys_stat = {}
for key in keys:
  keys_stat[key] = df[False == df[get_name_feat(key)].isnull()].shape[0] / df.shape[0] * 100

In [0]:
{k:v for k,v in keys_stat.items() if v > 27}

{'age group': 27.64770240700219,
 'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_age group_cat'] = df['feat_age group'].factorize()[0]
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]

df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]


for key in keys:
  df[get_name_feat(key) +'_cat'] = df[get_name_feat(key) ].factorize()[0]

In [0]:
df['brand'] = df['brand'].map(lambda x: str(x).lower() )
df [df.brand != df.feat_brand][['brand','feat_brand']].head()

,brand,feat_brand
21,rubies,generic
22,rubies,generic
23,rubies,generic
24,unbranded,NaN
31,american fighter,NaN


In [0]:
df [df.brand == df.feat_brand].shape

(8846, 532)

In [0]:
feats_cat=[x for x in df.columns if "cat" in x]

In [0]:
feats = ['feat_brand_cat','feat_color_cat','feat_gender_cat','feat_material_cat','feat_sport_cat','feat_style_cat']
#feats +=feats_cat
#feats = list(set(feats))

In [0]:
model1 = RandomForestRegressor( max_depth = 5,n_estimators=100, random_state = 0)
result = run_model(feats,model1)

In [0]:
X = df[feats].values
y=df['prices_amountmin'].values
m = RandomForestRegressor( max_depth = 5,n_estimators=100, random_state = 0)
m.fit(X,y)

print(result)
perm = PermutationImportance(m, random_state=1).fit(X,y);
eli5.show_weights(perm,feature_names=feats)


(-59.71519104663385, 4.029539373412338)


Weight,Feature
0.1029 ± 0.0056,feat_gender_cat
0.0942 ± 0.0014,feat_material_cat
0.0452 ± 0.0051,feat_style_cat
0.0405 ± 0.0024,feat_color_cat
0.0251 ± 0.0008,feat_brand_cat
0.0046 ± 0.0009,feat_sport_cat


In [0]:
df['brand'].value_counts()

nike                     1777
puma                      609
ralph lauren              526
vans                      386
new balance               371
                         ... 
roundtree & yorke           1
u.s. polo association       1
mr. durag                   1
firefly                     1
paul smith                  1
Name: brand, Length: 1732, dtype: int64

In [0]:
df['brand'].head()

0                  josmo
1                  josmo
2    servus by honeywell
3    servus by honeywell
4    servus by honeywell
Name: brand, dtype: object

In [0]:
df['brand_cat']

0           0
1           0
2           1
3           1
4           1
         ... 
18275    1731
18276     299
18277     399
18278     399
18279     399
Name: brand_cat, Length: 18280, dtype: int64

In [0]:
df['feat_brand_cat']

0          0
1          0
2          1
3          1
4          1
        ... 
18275     -1
18276    286
18277     -1
18278     -1
18279     -1
Name: feat_brand_cat, Length: 18280, dtype: int64

In [96]:
ls


data/       day5.ipynb         LICENSE      README.md
day4.ipynb  HelloGitHub.ipynb  matrix_one/


In [100]:
ls

day3.ipynb


In [0]:
!git add matrix_one/day5.ipynb

fatal: pathspec 'matrix_one/day5.ipynb' did not match any files
